In [1]:
#from google.colab import drive
import pandas as pd
import math
import random
#drive.mount('/content/drive')

#PIR COMPOSISITION
#------------3-----------
#----------0-1-2---------

In [2]:
silent_limit = 3
loitering_limit_day = 30
loitering_limit_night =30 
max_minimum_duration_day = 90
max_minimum_duration_night = 60

def time_update_string(the_sec, the_min, the_hour):
  cur_time = ""
  if(the_hour <10):
    cur_time = "0" + str(the_hour) + ":"
  else:
    cur_time = str(the_hour) + ":"
  if(the_min <10):
    cur_time += "0" + str(the_min) + ":"
  else:
    cur_time += str(the_min) + ":"
  if(the_sec <10):
    cur_time += "0" + str(the_sec)
  else:
    cur_time += str(the_sec)
  return cur_time

In [3]:
silent_counter = 0
cur_hour = 6
cur_min = 0
cur_sec = 0
pir_state = [0 for i in range(4)]
last_pir_state = [0 for i in range(4)]
for _lpr in range(4):
  last_pir_state[_lpr] = -1

file = open("/Users/asus/notebook/TA/PIR dataset Time train.csv","w")
file.write("Waktu,Pir_1,Pir_2,Pir_3,Pir_4,Gerakan\n")
file.close()
file = open("/Users/asus/notebook/TA/PIR dataset Duration train.csv","w")
file.write("Duration,Pir_1,Pir_2,Pir_3,Pir_4,Gerakan\n")
file.close()
total_data = 1036800
data_counter = 0

while(data_counter < total_data):                                                               #while amount of datas not required, keep generating datas
  if(True):                                                     
    if(random.randint(0,99) < 50):  #50% chance for movement                                    #xx% chance for movement data
      minimum_duration = random.randint(1,max_minimum_duration_day)         #generate duration of movements
      the_movement = ""
      duration = 0
      same_duration = 0
      prev_dif_pos = -1
      cur_pos = -1
      prev_pos = -1
      while(True):                          #while not moved out from ROI
        if(duration<minimum_duration):                #direction of step before reach minimum duration
          if(cur_pos == -1):
            cur_step = random.choice([1,3])
            last_pir_state[cur_pos + cur_step] = 1
          elif cur_pos == 0:                    
            cur_step = random.choice([0,1])
          elif cur_pos == 1:
            cur_step = random.choice([-1,0,1,2])
          elif cur_pos == 2:
            cur_step = random.choice([-1,0])
          elif cur_pos == 3:
            cur_step = random.choice([-2,0])
        else:                                         #after minimum duration reached so can moved out from ROI
          if cur_pos == 0:                    #determine direction of step
            cur_step = random.choice([-1,0,1])
          elif cur_pos == 1:
            cur_step = random.choice([-1,0,1,2])
          elif cur_pos == 2:
            cur_step = random.choice([-3,-1,0])
          else:
            cur_step = random.choice([-2,0])
        
        prev_pos = cur_pos                            #save previous position
        cur_pos += cur_step                           #save current position
        
        duration+=1

        if prev_pos == cur_pos:                       #check if it stay still
          silent_counter += 1
        else:
          silent_counter = 0
          prev_dif_pos = prev_pos

        if(prev_dif_pos == -1):           #if enter ROI
          pir_state[cur_pos] = 1
        elif(cur_pos == -1):              #if out of ROI
          for _p in range(4):
            pir_state[_p] = 0
        elif[prev_dif_pos != prev_pos]:   #if moved
          pir_state[prev_pos] = 0
          pir_state[cur_pos] = 1
        elif(prev_dif_pos == prev_pos):   #if stay still
          pir_state[cur_pos] = 1    

        if(True):                                     #Movement type determination
          # if silent_counter < silent_limit:
          #   if cur_pos == 0:
          #     if prev_dif_pos == 1:
          #       the_movement = "PERSON MOVE TOWARDS LEFT"
          #     elif prev_dif_pos == 0:
          #       the_movement = "STAY IN PLACE"
          #     else:
          #       the_movement = "PERSON MOVE TOWARDS RIGHT"
          #   elif cur_pos == 1:
          #     if prev_dif_pos == 0:
          #       the_movement = "PERSON MOVE TOWARDS RIGHT"
          #     elif prev_dif_pos == 2:
          #       the_movement = "PERSON MOVE TOWARDS LEFT"
          #     elif prev_dif_pos == 1:
          #       the_movement = "STAY IN PLACE"
          #     else:
          #       the_movement = "PERSON EXIT THE ROOM"
          #   elif cur_pos == 2:
          #     if prev_dif_pos == 1:
          #       the_movement = "PERSON MOVE TOWARDS RIGHT"
          #     elif prev_dif_pos == 2:
          #       the_movement = "STAY IN PLACE"
          #     else:
          #       the_movement = "PERSON MOVE TOWARDS LEFT"
          #   elif cur_pos == 3:
          #     if prev_dif_pos == 1:
          #       the_movement = "PERSON MOVE INSIDE THE ROOM"
          #     elif prev_dif_pos == 3:
          #       the_movement = "STAY IN PLACE"
          #     else:
          #       the_movement = "PERSON MOVE INSIDE THE ROOM"
          #   else:
          #     if prev_dif_pos == 0:
          #       the_movement = "PERSON MOVE TOWARDS LEFT"
          #     elif prev_dif_pos == 2:
          #       the_movement = "PERSON MOVE TOWARDS RIGHT"
          # elif silent_counter >= silent_limit:
          #   the_movement = "STAY IN PLACE"

          if duration >= loitering_limit_day:
            the_movement = "LOITERING"
          else:
            the_movement = "NORMAL"
        
        if(True):                                     #Write movement with duration
          match_mark = True
          for mm in range(4):
            if last_pir_state[mm] == pir_state[mm]:
              match_mark = False
          if match_mark == True:
            same_duration = 0
            file = open("/Users/asus/notebook/TA/PIR dataset Duration train.csv","a")
            file.write(str(same_duration))
            file.write(",")
            for _p in range(4):
              file.write(str(last_pir_state[_p]))
              file.write(",")
              last_pir_state[_p] = pir_state[_p]
            file.write(the_movement)
            file.write("\n")
            file.close()
          else:
            same_duration+=1

        if(True):                                     #Write movement with time
          file = open("/Users/asus/notebook/TA/PIR dataset Time train.csv","a")
          file.write(time_update_string(cur_sec, cur_min, cur_hour))
          file.write(",")
          for _p in range(4):
            file.write(str(pir_state[_p]))
            file.write(",")
          if cur_pos == -1 :
            file.write("NORMAL")
          else:
            file.write(the_movement)
          file.write("\n")
          file.close()

        if((cur_sec + 1) >59):                        #Update time
          cur_sec = 0
          if((cur_min + 1) >59):
            cur_min = 0
            if((cur_hour + 1) >24):
              cur_hour = 0
            else:
              cur_hour+=1
          else:
            cur_min+=1
        else:
          cur_sec+=1
        
        data_counter+=1
        if(cur_pos == -1):
          break
      if(True):
        file = open("/Users/asus/notebook/TA/PIR dataset Duration train.csv","a")
        file.write(str(same_duration))
        file.write(",")
        for _p in range(4):
          file.write(str(pir_state[_p]))
          file.write(",")
          last_pir_state[_p] = pir_state[_p]
        file.write(the_movement)
        file.write("\n")
        file.close()

    else:
      minimum_duration = random.randint(1,90)         #generate duration of movements
      the_movement = ""
      duration = 0
      same_duration = 0
      prev_dif_pos = -1
      cur_pos = -1
      prev_pos = -1
      while(duration <= minimum_duration):
        duration+=1
        file = open("/Users/asus/notebook/TA/PIR dataset Time train.csv","a")
        file.write(time_update_string(cur_sec, cur_min, cur_hour))
        file.write(",")
        for _p in range(4):
          file.write(str(pir_state[_p]))
          file.write(",")
        file.write("NORMAL")
        file.write("\n")
        file.close()
        if((cur_sec + 1) >59):                        #Update time
          cur_sec = 0
          if((cur_min + 1) >59):
            cur_min = 0
            if((cur_hour + 1) >24):
              cur_hour = 0
            else:
              cur_hour+=1
          else:
            cur_min+=1
        else:
          cur_sec+=1
      file = open("/Users/asus/notebook/TA/PIR dataset Duration train.csv","a")
      file.write(str(duration))
      file.write(",")
      file.write("0,0,0,0,")
      file.write("NORMAL")
      file.write("\n")
      file.close


In [4]:
train = pd.read_csv("PIR dataset Time train.csv")
print(train.shape)

(1941821, 6)


In [5]:
file = open("/Users/asus/notebook/TA/PIR dataset Time train.csv","r")
print(file.read())
file.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
file = open("/Users/asus/notebook/TA/PIR dataset Duration train.csv","w")
file.write("Durasi,Pir_1,Pir2,Pir3,Pir4,Gerakan\n")

datas = pd.read_csv("PIR dataset Time train.csv")
datas_size = datas.shape
datas_pir_state = [0 for i in range(4)]

pir_datas = datas.drop(["Waktu","Gerakan"], axis=1)
datas_movement = datas["Gerakan"]
match_duration = 0
stay_mark = False
# pin_mark = True

for a in range(4):
  datas_pir_state[a] = pir_datas.iloc[0][a]

for ds in range(datas_size[0]):
  pir_cnt = 0
  for pirdats in range(4):
    if datas_pir_state[pirdats] == pir_datas.iloc[ds][pirdats]:
      pir_cnt = pir_cnt + 1
  
  if pir_cnt == 4:
    match_duration+=1
  else:
    stay_mark= False
    file.write(str(match_duration))
    file.write(",")
    for tpd in range(4):
      file.write(str(datas_pir_state[tpd]))
      file.write(",")
      datas_pir_state[tpd] = pir_datas.iloc[ds][tpd]
    file.write(datas_movement[ds-1])
    file.write("\n")
    match_duration=1

  if (((match_duration == 4) and (stay_mark == False)) and (pir_cnt!=4)):
    stay_mark = True
    file.write(str(match_duration-1))
    file.write(",")
    for tpd in range(4):
      file.write(str(datas_pir_state[tpd]))
      file.write(",")
      datas_pir_state[tpd] = pir_datas.iloc[ds][tpd]
    file.write(datas_movement[ds-1])
    file.write("\n")
    match_duration=1


#last data
file.write(str(match_duration))
file.write(",")
for tpd in range(4):
  file.write(str(datas_pir_state[tpd]))
  file.write(",")
  datas_pir_state[tpd] = pir_datas.iloc[ds][tpd]
file.write(datas_movement[ds-1])
file.write("\n")
file.close()

In [8]:
file = open("/Users/asus/notebook/TA/PIR dataset Duration train.csv","r")
print(file.read())
file.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

